# Import moudle

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import pickle
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121,MobileNetV2,ResNet50V2,EfficientNetB3
import pandas as pd

# Build functions and class

In [ ]:
#Load dataset from directory
def Load_Dataset(generator,directory,img_height = 224, img_width = 224,batch_size = 64,class_mode = "categorical",shuffle = True):
  X = datagen.flow_from_directory(directory,
                                  target_size = (img_height,img_width),
                                  batch_size = batch_size,
                                  class_mode= class_mode,
                                  shuffle = shuffle)

  return X
def Load_All(datagen,train_path,test_path):
  test_generator = Load_Dataset(datagen,test_path)
  train_generator = Load_Dataset(datagen,train_path)
  return train_generator, test_generator
#Create class MixUp
class MixupImageDataGenerator():
    def __init__(self, generator, directory, batch_size = 64, img_height = 224, img_width = 224, alpha=0.2, subset=None):
        """Constructor for mixup image data generator.
        Arguments:
            generator {object} -- An instance of Keras ImageDataGenerator.
            directory {str} -- Image directory.
            batch_size {int} -- Batch size.
            img_height {int} -- Image height in pixels.
            img_width {int} -- Image width in pixels.
        Keyword Arguments:
            alpha {float} -- Mixup beta distribution alpha parameter. (default: {0.2})
            subset {str} -- 'training' or 'validation' if validation_split is specified in
            `generator` (ImageDataGenerator).(default: {None})
        """

        self.batch_index = 0
        self.batch_size = batch_size
        self.alpha = alpha

        # First iterator yielding tuples of (x, y)
        self.generator1 = generator.flow_from_directory(directory,
                                                        target_size=(
                                                            img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        subset=subset
                                                        )

        # Second iterator yielding tuples of (x, y)
        self.generator2 = generator.flow_from_directory(directory,
                                                        target_size=(
                                                            img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        subset=subset
                                                        )

        # Number of images across all classes in image directory.
        self.n = self.generator1.samples

    def reset_index(self):
        """Reset the generator indexes array.
        """

        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def on_epoch_end(self):
        self.reset_index()

    def reset(self):
        self.batch_index = 0

    def __len__(self):
        # round up
        return (self.n + self.batch_size - 1) // self.batch_size

    def get_steps_per_epoch(self):
        """Get number of steps per epoch based on batch size and
        number of images.
        Returns:
            int -- steps per epoch.
        """

        return self.n // self.batch_size

    def __next__(self):
        """Get next batch input/output pair.
        Returns:
            tuple -- batch of input/output pair, (inputs, outputs).
        """

        if self.batch_index == 0:
            self.reset_index()

        current_index = (self.batch_index * self.batch_size) % self.n
        if self.n > current_index + self.batch_size:
            self.batch_index += 1
        else:
            self.batch_index = 0

        # Get a pair of inputs and outputs from two iterators.
        X1, y1 = self.generator1.next()
        X2, y2 = self.generator2.next()

        # random sample the lambda value from beta distribution.
        l = np.random.beta(self.alpha, self.alpha, X1.shape[0])
        while 1.0 in l:
          l = np.random.beta(self.alpha, self.alpha, X1.shape[0])
        
        X_l = l.reshape(X1.shape[0], 1, 1, 1)
        y_l = l.reshape(y1.shape[0], 1)
        
        # Perform the mixup.
        X = X1 * X_l + X2 * (1 - X_l)
        y = y1 * y_l + y2 * (1 - y_l)

        return X, y

    def __iter__(self):
        while True:
            yield next(self)
#Merge more generator
def combine_gen(gens,bool = True):
  while True:
    for g in gens:
      if bool:
        yield next(g)
      else:
        img,lb = next(g)
        yield img*1./255,lb

In [ ]:
test_path = '/content/drive/MyDrive/HK5/Project_1/Đồ án số 1 khoa học dữ liệu/Datasets/Dataset Nhóm Mình/Test'
train_path = '/content/drive/MyDrive/HK5/Project_1/Đồ án số 1 khoa học dữ liệu/Datasets/Dataset Nhóm Mình/Train'

================================================================================

# Model  

In [ ]:
#Load Dense
def BuildModel(model_name,num_classes):
  #Change structure of model
  if model_name == "EfficientNetB3": model = EfficientNetB3()
  else: 
    if model_name == "ResNet50V2": model = ResNet50V2()
    else: 
      if model_name == "DenseNet121": model = DenseNet121()
      else: 
        if model_name == "MobileNetV2": model = MobileNetV2()
  base_inputs = model.layers[0].input
  base_outputs = model.layers[-2].output 
  final_outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(base_outputs)
  new_model = tf.keras.Model(inputs = base_inputs, outputs = final_outputs )
  #Compile
  m = tf.keras.metrics.TopKCategoricalAccuracy(k=5)
  opt = tf.keras.optimizers.Adam(learning_rate=5e-05)
  new_model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['accuracy', m] )
  new_model.summary()
  return new_model
def Training(train_generator,test_generator,model_name,step_epoch,num_classes = 15,epochs = 25,batch_size = 64):
  model = BuildModel(model_name,num_classes)
  checkpoint1 = tf.keras.callbacks.ModelCheckpoint(f"./Model/{model_name}/VNFood15/MixAug_{model_name}.h5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
  history = model.fit_generator(train_generator,
                                steps_per_epoch = step_epoch,            
                                epochs = epochs,
                                validation_data = test_generator,
                                validation_steps = test_generator.samples // batch_size,
                                callbacks=[checkpoint1])
  data = {'loss': history.history['loss'], 'accuracy': history.history['accuracy'],'top_5_categorical_accuracy':history.history['top_k_categorical_accuracy'], 'val_loss': history.history['val_loss'], 'val_acc': history.history['val_accuracy'],'val_top_k_categorical_accuracy':history.history['val_top_k_categorical_accuracy']}
  Csv = pd.DataFrame(data)
  Csv.to_csv(f'./History/{model_name}/VNFood15/MixAug_{model_name}_history.csv', index=0,header = False,mode = "a")

In [ ]:
%cd /content/drive/MyDrive/HK5/Project_1/Đồ án số 1 khoa học dữ liệu/DoAN2_Model
model_name = ["EfficientNetB3","ResNet50V2","DenseNet121","MobileNetV2"]
for i,name in enumerate(model_name):
  if i ==0:
    datagen = ImageDataGenerator()
    train_generator1,test_generator = Load_All(datagen,tradition_path,test_path)
    train_generator2 = MixupImageDataGenerator(datagen,train_path)
    train_generator3 = MixupImageDataGenerator(datagen,train_path)
    train_generator = combine_gen([train_generator3,train_generator2,train_generator1])
    step_epoch = len(train_generator1) + len(train_generator2) + len(train_generator3)
  else:
    if i == 2:
      datagen = ImageDataGenerator(rescale = 1./255)
      test_generator = Load_Dataset(datagen,test_path)
      train_generator = combine_gen([train_generator3,train_generator2,tradition_generator1],bool = False)
  Training(train_generator,test_generator,name,step_epoch)


/content/drive/.shortcut-targets-by-id/1y-8GHQhx9aBGJvqwl55KRe5aYhA4TEAN/Đồ án số 2 khoa học dữ liệu/DoAN2_Model
Found 6682 images belonging to 21 classes.
Found 30132 images belonging to 21 classes.
50102272/50095040 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 300, 300, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 300, 300, 3)  7           rescaling[0][0]                  
___________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
